In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
import yfinance as yf
from datetime import date
from data_preproc.data_preproc import clean_up_and_merge_dfs, process

In [2]:
df = pd.read_csv("data/spx.zip")
df["date"] = pd.to_datetime(df["date"])
df["exdate"] = pd.to_datetime(df["exdate"])
df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_6000\2635310518.py:1: DtypeWarning: Columns (23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/spx.zip")


,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag
0,108105,2000-01-03,09899.78,0,2000-01-22,1999-12-23,P,1505000,55.250,57.250,...,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN
1,108105,2000-01-03,098B5.3A,0,2000-01-22,NaN,P,1545000,89.625,91.625,...,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN
2,108105,2000-01-03,098C0.B7,0,2000-03-18,1999-12-29,C,750000,706.500,708.500,...,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN
3,108105,2000-01-03,098C4.73,0,2000-09-16,1999-12-27,C,1450000,135.750,137.750,...,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN
4,108105,2000-01-03,098D8.95,0,2000-06-17,2000-01-03,P,1350000,40.750,42.750,...,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,NaN


In [3]:
stock_price = pd.read_csv("data/GSPC.csv").rename(columns={"Date": "date"})
stock_price["date"] = pd.to_datetime(stock_price["date"])
stock_price["return"] = np.log(stock_price["Adj Close"]) - np.log(stock_price["Adj Close"].shift(1))
stock_price.head()

,date,Open,High,Low,Close,Adj Close,Volume,return
0,2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000,NaN
1,2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000,-0.039099
2,2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000,0.001920
3,2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000,0.000955
4,2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000,0.026730


In [4]:
df2 = clean_up_and_merge_dfs(df, stock_price)
df2.head()

[108105]
['P' 'C']
[1]
[0]
[1]
['CBOE S&P 500 INDEX']
['E']
[nan]
           date  next_date
2    2000-01-03 2000-01-04
520  2000-01-04 2000-01-05
1038 2000-01-05 2000-01-06
1564 2000-01-06 2000-01-07
2096 2000-01-07 2000-01-10
2638 2000-01-10 2000-01-11
3182 2000-01-11 2000-01-12
3728 2000-01-12 2000-01-13
4274 2000-01-13 2000-01-14
4822 2000-01-14 2000-01-18
               date  next_date
33693682 2023-02-14 2023-02-15
33712050 2023-02-15 2023-02-16
33730200 2023-02-16 2023-02-17
33748800 2023-02-17 2023-02-21
33766638 2023-02-21 2023-02-22
33783958 2023-02-22 2023-02-23
33802100 2023-02-23 2023-02-24
33820524 2023-02-24 2023-02-27
33838864 2023-02-27 2023-02-28
33857202 2023-02-28        NaT
3           256
6            74
7            46
8            18
12          256
           ... 
33875482    303
33875483    303
33875484    303
33875485    303
33875486    303
Name: day_to_maturity, Length: 14625542, dtype: int64


,date,symbol,symbol_flag,exdate,last_date,strike_price,best_bid,best_offer,volume,open_interest,...,Adj Close,Volume,return,index_pc_change,time_to_maturity,moneyness,actual_change,x1,x2,x3
0,2000-01-03,098C4.73,0,2000-09-16,1999-12-27,1450000,135.750,137.750,0,2365,...,1455.219971,931800000,NaN,NaN,1.024,0.621540,0.003465,NaN,NaN,NaN
1,2000-01-03,09908.AC,0,2000-03-18,2000-01-03,1575000,13.500,14.750,43,619,...,1455.219971,931800000,NaN,NaN,0.296,0.214505,-0.002984,NaN,NaN,NaN
2,2000-01-03,09917.26,0,2000-02-19,2000-01-03,1475000,35.375,37.375,550,3366,...,1455.219971,931800000,NaN,NaN,0.184,0.473627,0.005576,NaN,NaN,NaN
3,2000-01-03,09926.7A,0,2000-01-22,2000-01-03,1535000,2.000,2.750,77,77,...,1455.219971,931800000,NaN,NaN,0.072,0.093244,0.015423,NaN,NaN,NaN
4,2000-01-03,09962.45,0,2000-09-16,1999-12-20,1400000,167.875,169.875,0,36663,...,1455.219971,931800000,NaN,NaN,1.024,0.686076,0.004647,NaN,NaN,NaN


In [5]:
df2.to_parquet("data/cleaned_df.parquet", index=False)

In [6]:
df2 = pd.read_parquet("data/cleaned_df.parquet")

In [7]:
output = process(df2)
output.head()

2000-01-03 00:00:00
2000-01-04 00:00:00
2000-01-05 00:00:00
2000-01-06 00:00:00
2000-01-07 00:00:00
2000-01-10 00:00:00
2000-01-11 00:00:00
2000-01-12 00:00:00
2000-01-13 00:00:00
2000-01-14 00:00:00
2000-01-18 00:00:00
2000-01-19 00:00:00
2000-01-20 00:00:00
2000-01-21 00:00:00
2000-01-24 00:00:00
2000-01-25 00:00:00
2000-01-26 00:00:00
2000-01-27 00:00:00
2000-01-28 00:00:00
2000-01-31 00:00:00
2000-02-01 00:00:00
2000-02-02 00:00:00
2000-02-03 00:00:00
2000-02-04 00:00:00
2000-02-07 00:00:00
2000-02-08 00:00:00
2000-02-09 00:00:00
2000-02-10 00:00:00
2000-02-11 00:00:00
2000-02-14 00:00:00
2000-02-15 00:00:00
2000-02-16 00:00:00
2000-02-17 00:00:00
2000-02-18 00:00:00
2000-02-22 00:00:00
2000-02-23 00:00:00
2000-02-24 00:00:00
2000-02-25 00:00:00
2000-02-28 00:00:00
2000-02-29 00:00:00
2000-03-01 00:00:00
2000-03-02 00:00:00
2000-03-03 00:00:00
2000-03-06 00:00:00
2000-03-07 00:00:00
2000-03-08 00:00:00
2000-03-09 00:00:00
2000-03-10 00:00:00
2000-03-13 00:00:00
2000-03-14 00:00:00


,ttm_one_month_moneyness_pt_seven,ttm_one_month_moneyness_pt_eightfive,ttm_one_month_moneyness_pt_one,ttm_one_month_moneyness_pt_oneonefive,ttm_one_month_moneyness_pt_onethree,ttm_three_month_moneyness_pt_seven,ttm_three_month_moneyness_pt_eightfive,ttm_three_month_moneyness_pt_one,ttm_three_month_moneyness_pt_oneonefive,ttm_three_month_moneyness_pt_onethree,...,ttm_one_year_moneyness_pt_onethree,ttm_two_year_moneyness_pt_seven,ttm_two_year_moneyness_pt_eightfive,ttm_two_year_moneyness_pt_one,ttm_two_year_moneyness_pt_oneonefive,ttm_two_year_moneyness_pt_onethree,r_squared,mean_error,mean_absolute_error,observation
2000-01-03,6.194064e-02,0.312185,0.204530,0.130964,0.000000e+00,0.260440,0.275718,0.221527,0.168832,0.116171,...,0.158510,0.266024,0.271155,0.228732,0.205318,0.190285,0.995027,-2.735137e-06,0.001119,114.0
2000-01-04,1.363700e-01,0.320018,0.240132,0.144813,5.976089e-17,0.416818,0.300325,0.230710,0.182980,0.123392,...,0.174714,0.279396,0.281639,0.247485,0.215400,0.192983,0.997879,-1.101140e-06,0.001053,114.0
2000-01-05,9.418156e-18,0.342766,0.229619,0.144372,2.613037e-17,0.097694,0.295088,0.234404,0.188767,0.130037,...,0.173313,0.299431,0.283072,0.246802,0.217927,0.199251,0.998235,-1.896156e-07,0.001036,110.0
2000-01-06,1.702120e-16,0.154186,0.186524,0.144133,0.000000e+00,0.038355,0.251156,0.235466,0.164924,0.074584,...,0.165705,0.254652,0.274596,0.244797,0.213474,0.200466,0.980723,1.295322e-05,0.002518,107.0
2000-01-07,7.347157e-01,0.400736,0.205539,0.125236,4.878086e-17,0.393377,0.303629,0.220231,0.170690,0.167784,...,0.165183,0.294667,0.268464,0.237584,0.212596,0.192267,0.999162,-4.525523e-07,0.001423,128.0


In [8]:
output.to_parquet("data/spx_vol_surface_history_full_data_23.parquet")